In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

%matplotlib inline

In [ ]:
plt.plot()

## Prepare data

In [ ]:
x = loadmat('Signals_S')
y = loadmat("Labels_T")

In [ ]:
x = x['S']
y = y['T']

In [ ]:
x = x.T
y = y.reshape(200, -1)
y = y.T

Standardize inputs

In [ ]:
mean = np.mean(x, 0, keepdims=True)
std = np.std(x, 0, keepdims=True)

x = (x - mean) / std

In [ ]:
class DiffusionDataset(Dataset):
    def __init__(self, x, y):
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __getitem__(self, index):
        return self.x[index, :], self.y[index, :]

    def __len__(self):
        return self.x.shape[0]
    
def get_idx(n, train_val_test_fractions):
    index = np.arange(0, n)
    np.random.shuffle(index)

    train_val_test_fractions /= np.sum(train_val_test_fractions)
    split_locations = (np.cumsum(train_val_test_fractions[:-1]) * n).astype(int)
    train_idx, val_idx, test_idx = np.split(index, split_locations)
    return train_idx, val_idx, test_idx

In [ ]:
n = x.shape[0]
train_idx, val_idx, test_idx = get_idx(n, [0.8, 0.1, 0.1])

data_train = DiffusionDataset(x[train_idx, :], y[train_idx, :])
data_val = DiffusionDataset(x[val_idx, :], y[val_idx, :])
data_test = DiffusionDataset(x[test_idx, :], y[test_idx, :])

n_batch = 1000
loader_train = DataLoader(data_train, batch_size=n_batch, shuffle=True)
loader_val = DataLoader(data_val, batch_size=n_batch, shuffle=False)
loader_test = DataLoader(data_test, batch_size=n_batch, shuffle=False)

## Define neural network

In [ ]:
from torch.nn import ModuleList, Linear, LeakyReLU, MSELoss

class MLP(torch.nn.Module):

    def __init__(self, n_channels, act=LeakyReLU()):
      super().__init__()

      self.layers = torch.nn.ModuleList()
      for i in range(0, len(n_channels) - 1):
        if i > 0:
          self.layers.append(act)
        self.layers.append(Linear(n_channels[i], n_channels[i+1])) 

    def forward(self, x):
      for layer in self.layers:
        x = layer(x)
      return x

loss_fn = MSELoss()

In [ ]:
# To support GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
net = MLP([2000, 1000, 500, 200]) # First is input dim and last is output dim
net.to(device)

## Train

In [ ]:
loss_train = []
loss_val = []

optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3)


for epoch in range(2):

  net.train()
  running_loss = 0.0

  for batch_index, batch in enumerate(loader_train):
    x, y = batch
    x.to(device)
    y.to(device)
    
    pred = net(x)
    loss = loss_fn(pred, y) 
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  loss_train.append(running_loss / (batch_index + 1))

  net.eval()
  running_loss = 0.0
  for batch_index, batch in enumerate(loader_val):
    x, y = batch
    x.to(device)
    y.to(device)
    
    pred = net(x)      
    loss = loss_fn(pred, y) 

    running_loss += loss.item()

  loss_val.append(running_loss / (batch_index + 1))

In [ ]:
#plt.plot(loss_train)